In [13]:
#Basic imports
import math
import pandas as pd
import numpy as np
import re
from os import listdir
from os.path import isfile, join

# Tracking msg 

In [14]:
#Open files
trackFile = open('/home/sabrine/notebook/messages/trackMe.txt', 'r') 

In [15]:
# Split text file in order to extract different field
msg = trackFile.read().split('header: \n')[1:]

In [16]:
# Process thet tracking information
# Create different Series for the different information to easly process then later
number = []
seqs = []
secs = []
nsecs = []

# May be several id, height, distance, bounding box per sequence
ids_ = []
heights = []
distances = []
xs = []
ys = []
xs_bb = []
ys_bb = []
widths_bb = []
heights_bb = []

In [17]:
len(msg)


377

In [18]:
# Fill the Serie using appropriate regex to extract info
for i in range(len(msg)):
    # there should be one seq, sec, nsec and tracking information by track msg
    seq = re.search('[0-9]+',re.search('seq: [0-9]+', msg[i]).group(0)).group(0)
    sec = re.search('[0-9]+',re.search('stamp: \n    secs: [0-9]+', msg[i]).group(0)).group(0)
    nsec = re.search('[0-9]+',re.search('nsecs:\s+[0-9]+', msg[i]).group(0)).group(0)
    track = re.search('tracks:(?s).*', msg[i]).group().replace('tracks: ', '')
        
    # if severalAdvSearch('id: [0-9]+\n', track) is nan create a ligne with no trac info otherwise create a line with track info
    if not re.search('id: [0-9]+\n', track):
        # create row without track info
        #print('is nan')
        number.append(i)
        seqs.append(seq)
        secs.append(sec)
        nsecs.append(nsec)
        ids_.append(np.nan)
        heights.append(np.nan)
        distances.append(np.nan)
        xs.append(np.nan)
        ys.append(np.nan)
        xs_bb.append(np.nan)
        ys_bb.append(np.nan)
        widths_bb.append(np.nan)
        heights_bb.append(np.nan)
    else:
        #create rows with track info
        
        
        patternid = re.compile(r'(id: )([0-9]+)(\n)')
        patternheight = re.compile(r'(height: )([0-9]+\.[0-9]+)')
        patterndistance = re.compile(r'(distance: )([0-9]+\.[0-9]+)')
        patternx = re.compile(r'(box_2D: \n      x: )(-)?([0-9]+)(\n)') 
        patterny = re.compile(r'(y: )(-)?([0-9]+)(\n      width:)')
        patternxs = re.compile(r'(x: )([0-9]+\.[0-9]+)')
        patternys = re.compile(r'(y: )(-)?([0-9]+\.[0-9]+)')
        patternwidth = re.compile(r'(width: )([0-9]+)(\n)')
        patternheightbb = re.compile(r'(height: )([0-9]+)(\n)')
        
        #print('is not nan')
        #print('type', type(severalAdvSearch('id: [0-9]+\n', track)))
        
        for (letters, numbers, newline) in re.findall(patternid, track):
            number.append(i)
            seqs.append(seq)
            secs.append(sec)
            nsecs.append(nsec)
            ids_.append(numbers)
             #print letters, '*', numbers
                        
        for (letters, numbers) in re.findall(patternheight, track):
            heights.append(numbers)
            #print letters, '*', numbers
    
        for (letters, numbers) in re.findall(patterndistance, track):
            distances.append(numbers)
            #print letters, '*', numbers
            
        for (letters, sign,  numbers, newline) in re.findall(patternx, track):
            xs.append(sign + numbers)
            print letters, '*', numbers
            
        for (letters, sign, numbers, newline) in re.findall(patterny, track):
            ys.append(numbers)
            print letters, '*', numbers
            
        for (letters, numbers) in re.findall(patternxs, track):
            xs_bb.append(numbers)
            #print letters, '*', numbers
            
        for (letters, sign, numbers) in re.findall(patternys, track):
            ys_bb.append(numbers)
            #print letters, '*', numbers
           
        for (letters, numbers, newline) in re.findall(patternwidth, track):
            widths_bb.append(numbers)
            #print letters, '*', numbers
            
        for (letters, numbers, newline) in re.findall(patternheightbb, track):
            heights_bb.append(numbers)
            #print letters + 'bb', '*', numbers    
        #for g in severalAdvSearch('id: [0-9]+\n', track):
            #id_ = advSearch( '[0-9]+', g)
            #print ('yo', id_)
            #break

box_2D: 
      x:  * 270
box_2D: 
      x:  * 263
y:  * 110
y:  * 103
box_2D: 
      x:  * 261
y:  * 103
box_2D: 
      x:  * 256
y:  * 103
box_2D: 
      x:  * 252
y:  * 102
box_2D: 
      x:  * 244
y:  * 101
box_2D: 
      x:  * 237
y:  * 100
box_2D: 
      x:  * 229
y:  * 99
box_2D: 
      x:  * 222
y:  * 97
box_2D: 
      x:  * 218
y:  * 97
box_2D: 
      x:  * 210
y:  * 96
box_2D: 
      x:  * 207
y:  * 95
box_2D: 
      x:  * 198
y:  * 93
box_2D: 
      x:  * 191
y:  * 91
box_2D: 
      x:  * 186
y:  * 89
box_2D: 
      x:  * 185
y:  * 88
box_2D: 
      x:  * 180
y:  * 86
box_2D: 
      x:  * 180
y:  * 85
box_2D: 
      x:  * 175
y:  * 82
box_2D: 
      x:  * 172
y:  * 77
box_2D: 
      x:  * 170
y:  * 72
box_2D: 
      x:  * 171
y:  * 71
box_2D: 
      x:  * 170
y:  * 67
box_2D: 
      x:  * 171
y:  * 65
box_2D: 
      x:  * 171
y:  * 63
box_2D: 
      x:  * 171
y:  * 62
box_2D: 
      x:  * 171
y:  * 59
box_2D: 
      x:  * 170
y:  * 52
box_2D: 
      x:  * 172
y:  * 49
box_2D:

In [19]:
tra = pd.DataFrame(columns=['msgNumber', 'seq', 'secs', 'nsecs','id_', 'height', 'distance', 'x', 'y',
                            'x_bb', 'y_bb', 'width_bb', 'height_bb', 'time', 'frameMatch'])

In [20]:
len(xs)

1901

In [21]:
len(ys)

1901

In [22]:
len(ys_bb)

1901

In [23]:
tra.msgNumber = number
tra.seq = seqs
tra.secs = secs
tra.nsecs = nsecs
tra.id_ = ids_
tra.height = heights
tra.distance = distances
tra.x = xs
tra.y = ys
tra.x_bb = xs_bb
tra.y_bb = ys_bb
tra.width_bb = widths_bb
tra.height_bb = heights_bb

In [24]:
# Recreate the time of the detection in order to compare it to the frames' file name
timeDetec = (tra.secs + tra.nsecs).apply(pd.to_numeric).values.T.tolist()
tra.time = timeDetec

In [25]:
tra.head()

,msgNumber,seq,secs,nsecs,id_,height,distance,x,y,x_bb,y_bb,width_bb,height_bb,time,frameMatch
0,0,332,1493821804,107249000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1493821804107249000,NaN
1,1,333,1493821804,139261000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1493821804139261000,NaN
2,2,334,1493821804,183389000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1493821804183389000,NaN
3,3,335,1493821804,302242000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1493821804302242000,NaN
4,4,336,1493821804,350142000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1493821804350142000,NaN


In [26]:
#tra.id_ = tra.id_.astype(int)

ValueError: cannot convert float NaN to integer

In [ ]:
 #tra.msgNumber

In [27]:
tri = tra.set_index(['msgNumber','id_'])

In [28]:
tri.index.is_unique

True

# RGB images

In [29]:
# List all file from frames folder
mypath = "/home/sabrine/notebook/framesRGB"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [30]:
# Extract time from files' name
time_ = []
for t in onlyfiles:
    time_.append(int(re.search('^[0-9]*', t).group()))

In [31]:
len(time_)

388

# Synchronization

In [32]:
time_

[1493821816878415710,
 1493821805257795741,
 1493821810424532874,
 1493821811552327040,
 1493821812084325946,
 1493821812209125172,
 1493821808385691807,
 1493821817044022592,
 1493821821407949240,
 1493821817872742480,
 1493821833972028216,
 1493821808672171538,
 1493821815990399998,
 1493821827002834087,
 1493821811825803674,
 1493821821617889037,
 1493821805419897349,
 1493821811441986914,
 1493821814248442696,
 1493821817220224164,
 1493821827698216364,
 1493821833220029025,
 1493821812532802688,
 1493821810284059631,
 1493821819601811471,
 1493821810667021163,
 1493821833035933580,
 1493821815946236287,
 1493821824548803771,
 1493821822276123411,
 1493821805484226482,
 1493821820630949798,
 1493821832073118828,
 1493821833250998021,
 1493821820049440533,
 1493821807576143328,
 1493821827060603241,
 1493821825832654045,
 1493821814794686317,
 1493821825744681030,
 1493821828901167711,
 1493821809954834171,
 1493821807937588827,
 1493821823965590237,
 1493821819388571163,
 149382180

In [33]:
frame = []
i = 0
for msgNb, new_df in tri.groupby(level=0):
    for j in range(len(new_df)):
        frame.append(time_[i])
    i = i + 1

In [34]:
frame

[1493821816878415710,
 1493821805257795741,
 1493821810424532874,
 1493821811552327040,
 1493821812084325946,
 1493821812209125172,
 1493821808385691807,
 1493821817044022592,
 1493821821407949240,
 1493821817872742480,
 1493821833972028216,
 1493821808672171538,
 1493821815990399998,
 1493821827002834087,
 1493821811825803674,
 1493821821617889037,
 1493821805419897349,
 1493821811441986914,
 1493821814248442696,
 1493821817220224164,
 1493821827698216364,
 1493821833220029025,
 1493821812532802688,
 1493821810284059631,
 1493821819601811471,
 1493821810667021163,
 1493821833035933580,
 1493821815946236287,
 1493821824548803771,
 1493821822276123411,
 1493821805484226482,
 1493821820630949798,
 1493821832073118828,
 1493821832073118828,
 1493821833250998021,
 1493821820049440533,
 1493821807576143328,
 1493821827060603241,
 1493821825832654045,
 1493821814794686317,
 1493821825744681030,
 1493821828901167711,
 1493821809954834171,
 1493821807937588827,
 1493821823965590237,
 149382181

In [35]:
len(frame)

1901

In [36]:
tri.frameMatch = frame

In [37]:
tri.head(200)

,,seq,secs,nsecs,height,distance,x,y,x_bb,y_bb,width_bb,height_bb,time,frameMatch
msgNumber,id_,,,,,,,,,,,,,
0,NaN,332,1493821804,107249000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1493821804107249000,1493821816878415710
1,NaN,333,1493821804,139261000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1493821804139261000,1493821805257795741
2,NaN,334,1493821804,183389000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1493821804183389000,1493821810424532874
3,NaN,335,1493821804,302242000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1493821804302242000,1493821811552327040
4,NaN,336,1493821804,350142000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1493821804350142000,1493821812084325946
5,NaN,337,1493821804,400331000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1493821804400331000,1493821812209125172
6,NaN,338,1493821804,475117000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1493821804475117000,1493821808385691807
7,NaN,339,1493821804,572505000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1493821804572505000,1493821817044022592
8,NaN,340,1493821804,697255000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1493821804697255000,1493821821407949240


In [38]:
tri.to_pickle("/home/sabrine/social_tracking/syncPresentation")